methodology [here](https://hackmd.io/twOh6VgqSnyaAy8BbYrVLg?edit)

## installation and set up

In [ ]:
!pip install openai

from IPython.display import clear_output

clear_output()

from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key = OPENAI_API_KEY
)

client = OpenAI(api_key = OPENAI_API_KEY)

main_instructions = """
You are a helpful assistant that will be deployed for multiple different functions focused on generating narratives based on the Little Red Riding Hood tale.
You have access to a number of files, and you should retrieve passages from them to help guide your activity.
You also have access to a number of tools (functions to call), and it is important that you choose the right functions to use.
"""

assistant = client.beta.assistants.create(
    name="demo-bot-20240404-007",
    instructions=main_instructions,
    tools=[{"type": "retrieval"}],
    model="gpt-4-turbo-preview"
)

print(assistant)


Assistant(id='asst_PLymC0wPcU74yRa7vLP2BKh2', created_at=1712284911, description=None, file_ids=[], instructions='\nYou are a helpful assistant that will be deployed for multiple different functions focused on generating narratives based on the Little Red Riding Hood tale. \nYou have access to a number of files, and you should retrieve passages from them to help guide your activity.\nYou also have access to a number of tools (functions to call), and it is important that you choose the right functions to use.\n', metadata={}, model='gpt-4-turbo-preview', name='demo-bot-20240404-007', object='assistant', tools=[RetrievalTool(type='retrieval')])


In [ ]:
# upload files and store as `files`
import os

# Path to your assistant documents
assistant_docs_path = "/content/drive/MyDrive/ll_color_colabs/_people/_mk_color_colabs/_assets/cl200"

assistant_id=assistant.id

# Initialize an empty list to hold the file IDs
uploaded_file_ids = []

# Iterate over each file in the directory
for filename in os.listdir(assistant_docs_path):
    print(filename)
    file_path = os.path.join(assistant_docs_path, filename)
    print(file_path)

    # Check if it's a file (and not a directory)
    if os.path.isfile(file_path):
        print(f"Uploading {filename}...")
        # Upload the file and store the response
        response = client.files.create(
            file=open(file_path, "rb"),
            purpose="assistants"
        )

        print(response.dict())
        uploaded_file_ids.append(response.id)
        print(f"Uploaded {filename} and received id: {response.id}")

# Now, uploaded_file_ids contains all the uploaded file IDs
print("Uploaded file IDs:", uploaded_file_ids)




calvino-cybernetics-ghosts.pdf
/content/drive/MyDrive/ll_color_colabs/_people/_mk_color_colabs/_assets/cl200/calvino-cybernetics-ghosts.pdf
Uploading calvino-cybernetics-ghosts.pdf...
{'id': 'file-yF39qHIPDT94eKlTx34PttNf', 'bytes': 188262, 'created_at': 1712284990, 'filename': 'calvino-cybernetics-ghosts.pdf', 'object': 'file', 'purpose': 'assistants', 'status': 'processed', 'status_details': None}
Uploaded calvino-cybernetics-ghosts.pdf and received id: file-yF39qHIPDT94eKlTx34PttNf
rodari-grammar-of-fantasy.pdf
/content/drive/MyDrive/ll_color_colabs/_people/_mk_color_colabs/_assets/cl200/rodari-grammar-of-fantasy.pdf
Uploading rodari-grammar-of-fantasy.pdf...
{'id': 'file-NkVJKljpA4cFxQywI2Ll0L09', 'bytes': 75456824, 'created_at': 1712284995, 'filename': 'rodari-grammar-of-fantasy.pdf', 'object': 'file', 'purpose': 'assistants', 'status': 'processed', 'status_details': None}
Uploaded rodari-grammar-of-fantasy.pdf and received id: file-NkVJKljpA4cFxQywI2Ll0L09
jakobson-folklore-creat

In [ ]:
# update assistant with retrieval

import datetime
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# from previous step in this colab
file_ids = uploaded_file_ids
new_assisant_name = f"demo-bot-w-retrieval-{timestamp}"


my_updated_assistant = client.beta.assistants.update(
  assistant.id,
  name=new_assisant_name,
  # instructions="you are a ",
  tools=[{"type": "retrieval"}],
  model="gpt-4-turbo-preview",
  file_ids=file_ids,
)

print(my_updated_assistant)

Assistant(id='asst_PLymC0wPcU74yRa7vLP2BKh2', created_at=1712284911, description=None, file_ids=['file-yF39qHIPDT94eKlTx34PttNf', 'file-NkVJKljpA4cFxQywI2Ll0L09', 'file-zr9KKbkone3QUyD55QG133uM', 'file-LgW9EgEoETWCE6ZL8zA2AXLE', 'file-KG2YYqAvKRhSTaE7FdrKEjXQ', 'file-J7KVhY3VmGFYhR6t57xBtOcv', 'file-3LKjbH3uK602H5qSQ9sqdjod'], instructions='\nYou are a helpful assistant that will be deployed for multiple different functions focused on generating narratives based on the Little Red Riding Hood tale. \nYou have access to a number of files, and you should retrieve passages from them to help guide your activity.\nYou also have access to a number of tools (functions to call), and it is important that you choose the right functions to use.\n', metadata={}, model='gpt-4-turbo-preview', name='demo-bot-w-retrieval-20240405024327', object='assistant', tools=[RetrievalTool(type='retrieval')])


In [ ]:
# update assistant with function calling

import datetime
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# let's update the name again just to help us track our progress
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
new_assisant_name = f"demo-bot-w-functions-{timestamp}"

my_function_calling_assistant = client.beta.assistants.update(
  assistant.id,
  name=new_assisant_name,
  tools=[
      {"type": "retrieval"},
      {
        "type": "function",
        "function": {
            "name": "generate_narrative_structure",
            "description": "Generates a narrative structure based on selected narratemes and stages.",
            "parameters": {
                "type": "object",
                "properties": {
                    "color_theme": {
                        "type": "string",
                        "enum": [
                          "AliceBlue",
                          "AntiqueWhite",
                          "Aqua",
                          "Aquamarine",
                          "Azure",
                          "Beige",
                          "Bisque",
                          "Black",
                          "BlanchedAlmond",
                          # "Blue",
                          "BlueViolet",
                          "Brown",
                          "BurlyWood",
                          "CadetBlue",
                          "Chartreuse",
                          "Chocolate",
                          "Coral",
                          "CornflowerBlue",
                          "Cornsilk",
                          "Crimson",
                          "Cyan",
                          "DarkBlue",
                          "DarkCyan",
                          "DarkGoldenRod",
                          "DarkGray",
                          "DarkGreen",
                          "DarkKhaki",
                          "DarkMagenta",
                          "DarkOliveGreen",
                          "DarkOrange",
                          "DarkOrchid",
                          "DarkRed",
                          "DarkSalmon",
                          "DarkSeaGreen",
                          "DarkSlateBlue",
                          "DarkSlateGray",
                          "DarkTurquoise",
                          "DarkViolet",
                          "DeepPink",
                          "DeepSkyBlue",
                          "DimGray",
                          "DodgerBlue",
                          "FireBrick",
                          "FloralWhite",
                          "ForestGreen",
                          "Fuchsia",
                          "Gainsboro",
                          "GhostWhite",
                          "Gold",
                          "GoldenRod",
                          "Gray",
                          "Green",
                          "GreenYellow",
                          "HoneyDew",
                          "HotPink",
                          "IndianRed",
                          "Indigo",
                          "Ivory",
                          "Khaki",
                          "Lavender",
                          "LavenderBlush",
                          "LawnGreen",
                          "LemonChiffon",
                          "LightBlue",
                          "LightCoral",
                          "LightCyan",
                          "LightGoldenRodYellow",
                          "LightGray",
                          "LightGreen",
                          "LightPink",
                          "LightSalmon",
                          "LightSeaGreen",
                          "LightSkyBlue",
                          "LightSlateGray",
                          "LightSteelBlue",
                          "LightYellow",
                          "Lime",
                          "LimeGreen",
                          "Linen",
                          "Magenta",
                          "Maroon",
                          "MediumAquaMarine",
                          "MediumBlue",
                          "MediumOrchid",
                          "MediumPurple",
                          "MediumSeaGreen",
                          "MediumSlateBlue",
                          "MediumSpringGreen",
                          "MediumTurquoise",
                          "MediumVioletRed",
                          "MidnightBlue",
                          "MintCream",
                          "MistyRose",
                          "Moccasin",
                          "NavajoWhite",
                          "Navy",
                          "OldLace",
                          "Olive",
                          "OliveDrab",
                          "Orange",
                          "OrangeRed",
                          "Orchid",
                          "PaleGoldenRod",
                          "PaleGreen",
                          "PaleTurquoise",
                          "PaleVioletRed",
                          "PapayaWhip",
                          "PeachPuff",
                          "Peru",
                          "Pink",
                          "Plum",
                          "PowderBlue",
                          "Purple",
                          "RebeccaPurple",
                          "Red",
                          "RosyBrown",
                          "RoyalBlue",
                          "SaddleBrown",
                          "Salmon",
                          "SandyBrown",
                          "SeaGreen",
                          "SeaShell",
                          "Sienna",
                          "Silver",
                          "SkyBlue",
                          "SlateBlue",
                          "SlateGray",
                          "Snow",
                          "SpringGreen",
                          "SteelBlue",
                          "Tan",
                          "Teal",
                          "Thistle",
                          "Tomato",
                          "Turquoise",
                          "Violet",
                          "Wheat",
                          "White",
                          "WhiteSmoke",
                          "Yellow",
                          "YellowGreen"
                        ],
                        "description": "The color theme, influencing the narrative's mood and elements, referencing Goethe's color theory.",
                    },
                    "narratemes": {
                        "type": "array",
                        "items": {
                            "type": "string",
                            "enum": [
                                "absentation", "interdiction", "violation", "reconnaissance",
                                "delivery", "trickery", "complicity", "villainy", "mediation",
                                "beginning counteraction", "departure", "first function of the donor",
                                "hero's reaction", "receipt of a magical agent", "guidance", "struggle",
                                "branding", "victory", "liquidation", "return", "pursuit", "rescue",
                                "unrecognized arrival", "unfounded claims", "difficult task",
                                "solution", "recognition", "exposure", "transfiguration",
                                "punishment", "wedding"
                            ],
                            "description": "Select the 5-10 most relevant of Propp's narratemes to include in the narrative structure.",
                        },
                        "minItems": 1,
                        "uniqueItems": True
                    },
                    "stages": {
                        "type": "array",
                        "items": {
                            "type": "string",
                            "enum": [
                                "call to adventure", "refusal of the call", "supernatural aid",
                                "crossing the first threshold", "belly of the whale", "the road of trials",
                                "meeting with the goddess", "woman as temptress", "atonement with the father",
                                "apotheosis", "the ultimate boon", "refusal of the return",
                                "the magic flight", "rescue from without", "the crossing of the return threshold",
                                "master of two worlds", "freedom to live"
                            ],
                            "description": "Select at least 3 of the most relevant of Campbell's stages to weave into the narrative structure.",
                        },
                        "minItems": 1,
                        "uniqueItems": True
                    }
                },
                "required": ["color_theme", "narratemes", "stages"]
            }
        }
    }
  ],
)


In [ ]:
thread = client.beta.threads.create()

print(thread)

Thread(id='thread_FanPXGWeq0lQgwKk5Jz6cU5e', created_at=1712285845, metadata={}, object='thread')


In [ ]:
from openai import OpenAI
client = OpenAI(api_key = OPENAI_API_KEY)

thread_message = client.beta.threads.messages.create(
  thread.id,
  role="user",
  content="please write me a story about little rgb = 50, 75, 195 riding hood",
)
print(thread_message.dict())

{'id': 'msg_ZRnGEedTNQX24fExa0fLEFPi', 'assistant_id': None, 'completed_at': None, 'content': [{'text': {'annotations': [], 'value': 'please write me a story about little rgb = 50, 75, 195 riding hood'}, 'type': 'text'}], 'created_at': 1712285847, 'file_ids': [], 'incomplete_at': None, 'incomplete_details': None, 'metadata': {}, 'object': 'thread.message', 'role': 'user', 'run_id': None, 'status': None, 'thread_id': 'thread_FanPXGWeq0lQgwKk5Jz6cU5e'}


In [ ]:
import json

print(json.dumps(thread_message.dict(), indent=4  ))

{
    "id": "msg_ZRnGEedTNQX24fExa0fLEFPi",
    "assistant_id": null,
    "completed_at": null,
    "content": [
        {
            "text": {
                "annotations": [],
                "value": "please write me a story about little rgb = 50, 75, 195 riding hood"
            },
            "type": "text"
        }
    ],
    "created_at": 1712285847,
    "file_ids": [],
    "incomplete_at": null,
    "incomplete_details": null,
    "metadata": {},
    "object": "thread.message",
    "role": "user",
    "run_id": null,
    "status": null,
    "thread_id": "thread_FanPXGWeq0lQgwKk5Jz6cU5e"
}


In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant_id,
)

print(run.dict())



{'id': 'run_vz6GGLc4BkMu5XjORtHji9jk', 'assistant_id': 'asst_PLymC0wPcU74yRa7vLP2BKh2', 'cancelled_at': None, 'completed_at': None, 'created_at': 1712286105, 'expires_at': 1712286705, 'failed_at': None, 'file_ids': ['file-yF39qHIPDT94eKlTx34PttNf', 'file-NkVJKljpA4cFxQywI2Ll0L09', 'file-zr9KKbkone3QUyD55QG133uM', 'file-LgW9EgEoETWCE6ZL8zA2AXLE', 'file-KG2YYqAvKRhSTaE7FdrKEjXQ', 'file-J7KVhY3VmGFYhR6t57xBtOcv', 'file-3LKjbH3uK602H5qSQ9sqdjod'], 'instructions': '\nYou are a helpful assistant that will be deployed for multiple different functions focused on generating narratives based on the Little Red Riding Hood tale. \nYou have access to a number of files, and you should retrieve passages from them to help guide your activity.\nYou also have access to a number of tools (functions to call), and it is important that you choose the right functions to use.\n', 'last_error': None, 'metadata': {}, 'model': 'gpt-4-turbo-preview', 'object': 'thread.run', 'required_action': None, 'started_at': 

In [ ]:
run_id=run.id

run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run_id
)

# print(json.dumps(run.dict(), indent=4))
print(run.status)


completed


In [ ]:
print(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments)

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [ ]:
# optional: do something with the arguments (i.e. run the function)

response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"help me create a prompt for chatgpt that will generate a fairy tale with these strict parameters: {run.required_action.submit_tool_outputs.tool_calls[0].function.arguments}. Please change the riding-hood's name to match the color theme supplied. And ensure that the story aligns with the feel of this theme as Goethe would have argued, based on the Goethe text on color theory you can retrieve from."}
  ]
)

print(response.choices[0].message.content)

KeyboardInterrupt: 

In [ ]:
# send back results of function

run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=[
      {
        "tool_call_id": run.required_action.submit_tool_outputs.tool_calls[0].id,
        "output": f"the story must adhere to these parameters from the readings you've been trained on, and then list them at the bottom: {run.required_action.submit_tool_outputs.tool_calls[0].function.arguments}. Please do not include these words in the story. Just list them at the bottom. Also change the riding hood's name to match the color theme supplied, and make the elements of the story match the mood of that color, as you would guess given where it would fit into the theory of color Goethe provides in the text you have access to via the retrieval tool."
      }
  ]
)


AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [ ]:
print(run.status)

completed


In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages)



SyncCursorPage[Message](data=[Message(id='msg_8lp7VGGIwXfy9Q8PsJEZmSuz', assistant_id='asst_PLymC0wPcU74yRa7vLP2BKh2', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='In the heart of a dense, twilight forest, where the canopy wove a vault of DarkSlateBlue, there lived a young girl known to all as Little DarkSlateBlue Riding Hood. A cloak the color of deep, night sky draped over her shoulders, shimmering with hints of a distant storm. This garment, a gift from her grandmother, was said to possess enchanting properties, wrapping the wearer in the embrace of mystery and adventure.\n\nOne crisp evening, as the last light wavered on the horizon, Little DarkSlateBlue Riding Hood received a call to adventure. Her grandmother, residing on the other side of the forest, had fallen ill, and only the healing berries of the Eldergloom could restore her health. Gathering her courage like a cloak around her, Little DarkSlateBlue Riding Hood departed, stepping into the sh

In [ ]:
print(json.dumps(messages.dict(), indent=4))

{
    "data": [
        {
            "id": "msg_8lp7VGGIwXfy9Q8PsJEZmSuz",
            "assistant_id": "asst_PLymC0wPcU74yRa7vLP2BKh2",
            "completed_at": null,
            "content": [
                {
                    "text": {
                        "annotations": [],
                        "value": "In the heart of a dense, twilight forest, where the canopy wove a vault of DarkSlateBlue, there lived a young girl known to all as Little DarkSlateBlue Riding Hood. A cloak the color of deep, night sky draped over her shoulders, shimmering with hints of a distant storm. This garment, a gift from her grandmother, was said to possess enchanting properties, wrapping the wearer in the embrace of mystery and adventure.\n\nOne crisp evening, as the last light wavered on the horizon, Little DarkSlateBlue Riding Hood received a call to adventure. Her grandmother, residing on the other side of the forest, had fallen ill, and only the healing berries of the Eldergloom could restor

In [ ]:
print(messages.data[0].content[0].text.value)


In the heart of a dense, twilight forest, where the canopy wove a vault of DarkSlateBlue, there lived a young girl known to all as Little DarkSlateBlue Riding Hood. A cloak the color of deep, night sky draped over her shoulders, shimmering with hints of a distant storm. This garment, a gift from her grandmother, was said to possess enchanting properties, wrapping the wearer in the embrace of mystery and adventure.

One crisp evening, as the last light wavered on the horizon, Little DarkSlateBlue Riding Hood received a call to adventure. Her grandmother, residing on the other side of the forest, had fallen ill, and only the healing berries of the Eldergloom could restore her health. Gathering her courage like a cloak around her, Little DarkSlateBlue Riding Hood departed, stepping into the shadow-laden paths of the forest.

Yet, the forest was not uninhabited. Villainy lurked in the guise of a sly, cunning wolf whose fur was as dark as the chasms of the night. He had heard of the girl's 

In [ ]:
# or load assistant



### thread

only one thread is needed. after this PROMPT A and B can simply be different messages referencing the same thread.

In [ ]:
thread = client.beta.threads.create()

# Create a message in the thread asking to create a narrative skeleton in JSON
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # Use dot notation here instead of ['id']
    role="user",
    content="Generate a Little Red Riding Hood tale narrative skeleton in JSON format, including logically ordered beats of narrative plot points, remixing Propp's 31 Narratemes and Campbell's 12 Stages. They should, in order, cover the basic three-part structure of Little Red Riding Hood (the journey, the encounter with the wolf, and the rescue). However, multiple narratemes/steps, up to ten, can be combined innovatively to make and enrich this basic three-part plot."
)

print(thread)


Thread(id='thread_8Q4xVY5TsjvrGCSrat3tdmPn', created_at=1712180764, metadata={}, object='thread')
